In [22]:
import time
import sagemaker
from sagemaker.sklearn import SKLearn
from sagemaker.experiments.experiment import Experiment
from sagemaker import get_execution_role

# Define S3 bucket and prefix
s3_bucket = 'sagemaker-bucket-ds'
file_path = '01_STOCKS/DATA/JOINED/joined_dataset.csv'

# Initialize SageMaker session
session = sagemaker.Session()

# Function to get or create an experiment
def get_or_create_experiment(experiment_name, description):
    try:
        experiment = Experiment.load(experiment_name=experiment_name, sagemaker_session=session)
        print(f"Experiment '{experiment_name}' already exists.")
    except Exception as e:
        if 'ValidationException' in str(e) and 'already exists' in str(e):
            experiment = Experiment.load(experiment_name=experiment_name, sagemaker_session=session)
            print(f"Experiment '{experiment_name}' already exists.")
        else:
            print(f"Creating experiment '{experiment_name}'.")
            experiment = Experiment.create(
                experiment_name=experiment_name,
                description=description
            )
    return experiment

# Create or get the experiment
experiment_name = 'linear-regression-experiment5'
experiment_description = "Experiment for Linear Regression Model"
experiment = get_or_create_experiment(experiment_name, experiment_description)

role = get_execution_role()

# Define the estimator
sklearn_estimator = SKLearn(
    entry_point='train.py',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large', # This is the smallest instance possible
    framework_version='0.23-1',
    base_job_name='sklearn-linear-regression',
    enable_sagemaker_metrics=True,
    experiment_config={
        "ExperimentName": experiment.experiment_name,
        "TrialName": f"trial-{int(time.time())}",
        "TrialComponentDisplayName": "Training"
    }
)

# Fit the model
sklearn_estimator.fit({'train': f's3://{s3_bucket}/{file_path}'})

Experiment 'linear-regression-experiment5' already exists.


INFO:sagemaker:Creating training-job with name: sklearn-linear-regression-2024-06-14-10-52-10-396


2024-06-14 10:52:10 Starting - Starting the training job...
2024-06-14 10:52:31 Starting - Preparing the instances for training...
2024-06-14 10:52:58 Downloading - Downloading input data......
2024-06-14 10:53:44 Downloading - Downloading the training image..2024-06-14 10:54:23,515 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-06-14 10:54:23,518 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-14 10:54:23,557 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-06-14 10:54:23,711 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-14 10:54:23,723 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-14 10:54:23,734 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-14 10:54:23,743 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    

UnexpectedStatusException: Error for Training job sklearn-linear-regression-2024-06-14-10-52-10-396: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 35, in train
    runner_type=runner.ProcessRunnerType)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/entry_point.py", line 100, in run
    wait, capture_error
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 291, in run
    cwd=environment.code_dir,
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 208, in check_error
    info=extra_info,
sagemaker_training.error